<a href="https://colab.research.google.com/github/adalves-ufabc/2025.Q3-PLN/blob/main/2025_Q3_PLN_AULA_17_Notebook_32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2025-Q3]**
Prof. Alexandre Donizeti Alves

## **Geração de Dados Sintéticos**
---



**Dados sintéticos** são dados gerados artificialmente, em vez de dados coletados de eventos do mundo real. São usados para simular dados reais sem comprometer a privacidade ou enfrentar limitações do mundo real.



Benefícios dos Dados Sintéticos:

- **Privacidade e Segurança**: sem dados pessoais reais em risco de vazamentos.
- **Aumento de Dados**: expande conjuntos de dados para aprendizado de máquina.
- **Flexibilidade**: cria cenários específicos ou raros.
- **Custo-efetivo**: muitas vezes mais barato do que a coleta de dados reais.
- **Conformidade Regulatória**: ajuda a navegar em leis rigorosas de proteção de dados.
- **Robustez do Modelo**: pode levar a modelos de IA que generalizam melhor.
- **Prototipagem Rápida**: permite testes rápidos sem dados reais.
- **Experimentação Controlada**: simula condições específicas.
- **Acesso a Dados**: alternativa quando dados reais não estão disponíveis.

**NOTA**: apesar dos benefícios, os dados sintéticos devem ser usados com cuidado, pois podem não capturar sempre as complexidades do mundo real.

#### **LangChain**

In [ ]:
#@title Instalando o pacote LangChain
!pip install -qU langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 7.9 MB/s eta 0:00:00


In [ ]:
#@title Versão do LangChain

import langchain

print(langchain.__version__)

1.0.5


In [ ]:
#@title Integração com o pacote da OpenAI

!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.2 MB/s eta 0:00:00


In [ ]:
#@title Definindo a chave da API da OpenAI

import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
#@title Definindo a chave da API da OpenAI
from getpass import getpass

OPENAI_API_KEY = getpass()

··········


#### **Exemplo**

Neste notebook, vamos nos aprofundar na geração de registros de faturamento médico sintéticos usando a biblioteca `LangChain`. Esta ferramenta é particularmente útil quando você deseja desenvolver ou testar algoritmos, mas não quer usar dados reais de pacientes devido a preocupações com a privacidade ou problemas de disponibilidade de dados.



**Configuração**
>

Primeiro, você precisará ter a biblioteca `LangChain` instalada, juntamente com suas dependências. Como estamos usando a cadeia de geradores da OpenAI, também a instalaremos. Como esta é uma biblioteca experimental, precisaremos incluir `langchain_experimental` em nossas instalações.

In [ ]:
!pip install -qU langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.6/209.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from pydantic import BaseModel
from langchain_experimental.tabular_synthetic_data.openai import (
    OPENAI_TEMPLATE,
    create_openai_data_generator,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)
from langchain_openai import ChatOpenAI

**1. Defina Seu Modelo de Dados**

>
Cada conjunto de dados tem uma estrutura ou um `esquema`. A classe `FaturamentoMedico` abaixo serve como nosso esquema para os dados sintéticos. Ao definir isso, estamos informando nosso gerador de dados sintéticos sobre a forma e a natureza dos dados que esperamos.

In [ ]:
class FaturamentoMedico(BaseModel):
    id_paciente: int
    nome_paciente: str
    codigo_diagnostico: str
    codigo_procedimento: str
    custo_total: float
    valor_reclamacao_seguro: float

**2. Amostra de Dados**
>

Para orientar o gerador de dados sintéticos, é útil fornecer alguns exemplos que se assemelhem ao mundo real. Esses exemplos servem como uma "semente" - eles são representativos do tipo de dados que você deseja, e o gerador os usará para criar mais dados que se pareçam com eles.

>
Aqui estão alguns registros fictícios de faturamento médico:

In [ ]:
exemplos = [
    {
        "example": """ID Paciente: 123456, Nome Paciente: John Doe, Código Diagnóstico:
        J20.9, Código Procedimento: 99203, Custo Total: R$2500, Valor reclamação seguro: R$1350"""
    },
    {
        "example": """ID Paciente: 789012, Nome Pacientee: Johnson Smith, Código Diagnóstico:
        M54.5, Código Procedimento: 99213, Custo Total: R$1515, Valor reclamação seguro: R$820"""
    },
    {
        "example": """ID Paciente: 345678, Nome Paciente: Emily Stone, Código Diagnóstico:
        E11.9, Código Procedimento: 99214, Custo Total: R$3300, Valor reclamação seguro: R$2250"""
    },
]

In [ ]:
exemplos

[{'example': 'ID Paciente: 123456, Nome Paciente: John Doe, Código Diagnóstico:\n        J20.9, Código Procedimento: 99203, Custo Total: R$2500, Valor reclamação seguro: R$1350'},
 {'example': 'ID Paciente: 789012, Nome Pacientee: Johnson Smith, Código Diagnóstico:\n        M54.5, Código Procedimento: 99213, Custo Total: R$1515, Valor reclamação seguro: R$820'},
 {'example': 'ID Paciente: 345678, Nome Paciente: Emily Stone, Código Diagnóstico:\n        E11.9, Código Procedimento: 99214, Custo Total: R$3300, Valor reclamação seguro: R$2250'}]

**3. Crie um Template de Prompt**
>
O gerador não sabe magicamente como criar nossos dados; precisamos guiá-lo. Fazemos isso criando um template de *prompt*. Esse template ajuda a instruir o modelo de linguagem subjacente sobre como produzir dados sintéticos no formato desejado.

In [ ]:
OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix = SYNTHETIC_FEW_SHOT_PREFIX,
    examples = exemplos,
    suffix = SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables = ["subject", "extra"],
    example_prompt = OPENAI_TEMPLATE,
)

O `FewShotPromptTemplate` inclui:

- **`prefix` e `suffix`**: estes provavelmente contêm contexto ou instruções orientadoras.
- **`examples`**: os dados de amostra que definimos anteriormente.
- **`input_variables`**: essas variáveis (`"subject"`, `"extra"`) são espaços reservados que você pode preencher dinamicamente mais tarde. Por exemplo, `"subject"` pode ser preenchido com `"faturamento_medico"` para orientar ainda mais o modelo.
- **`example_prompt`**: este modelo de *prompt* é o formato que queremos que cada linha de exemplo tenha em nosso *prompt*.

**4. Criando o Gerador de Dados**

>
Com o esquema e o *prompt* prontos, o próximo passo é criar o gerador de dados. Este objeto sabe como se comunicar com o modelo de linguagem subjacente para obter dados sintéticos.

In [ ]:
gerador_dados = create_openai_data_generator(
    output_schema = FaturamentoMedico,
    llm = ChatOpenAI(
        temperature = 1,
        api_key = OPENAI_API_KEY
    ),
    prompt = prompt_template,
)

**5. Gerar Dados Sintéticos**

>
Finalmente, vamos obter nossos dados sintéticos!

In [ ]:
resposta = gerador_dados.generate(
    subject="faturamento_medico",
    extra="o nome deve ser escolhido aleatoriamente. Escolha algo que você normalmente não escolheria.",
    runs=10
)

Este comando solicita ao gerador que produza 10 registros sintéticos de faturamento médico. A saída será uma lista dos modelos Pydantic `FaturamentoMedico`.

In [ ]:
resposta

[FaturamentoMedico(id_paciente=987654, nome_paciente='Luna Rodriguez', codigo_diagnostico='F32.9', codigo_procedimento='99204', custo_total=2800.0, valor_reclamacao_seguro=1500.0),
 FaturamentoMedico(id_paciente=123456, nome_paciente='Xander Thompson', codigo_diagnostico='G47.0', codigo_procedimento='99203', custo_total=2100.0, valor_reclamacao_seguro=1200.0),
 FaturamentoMedico(id_paciente=543210, nome_paciente='Aurora Patel', codigo_diagnostico='K72.1', codigo_procedimento='99213', custo_total=2500.0, valor_reclamacao_seguro=1350.0),
 FaturamentoMedico(id_paciente=246810, nome_paciente='Ezekiel Morales', codigo_diagnostico='I25.10', codigo_procedimento='99214', custo_total=2900.0, valor_reclamacao_seguro=1600.0),
 FaturamentoMedico(id_paciente=987654, nome_paciente='Octavia Singh', codigo_diagnostico='F32.9', codigo_procedimento='99204', custo_total=2750.0, valor_reclamacao_seguro=1400.0),
 FaturamentoMedico(id_paciente=123456, nome_paciente='Xander Chang', codigo_diagnostico='J44.9'

**Referência:**

> https://medium.com/@danushidk507/synthetic-data-generation-with-langchain-d4217d5dea74

>